In [1]:
import sys
import grpc
import zemberek_grpc.morphology_pb2 as z_morphology
import zemberek_grpc.morphology_pb2_grpc as z_morphology_g
import pandas as pd
from datetime import datetime
import re
from nltk.corpus import stopwords
import string

channel = grpc.insecure_channel('localhost:6789')

morphology_stub = z_morphology_g.MorphologyServiceStub(channel)

df=pd.read_csv("csvFiles\\thy_news.csv",index_col=0)

In [2]:
def analyze(i):
    response = morphology_stub.AnalyzeSentence(z_morphology.SentenceAnalysisRequest(input=i))
    return response

In [3]:
def fix_decode(text):
    """Pass decode."""
    if sys.version_info < (3, 0):
        return text.decode('utf-8')
    else:
        return text

In [ ]:
def zemberek(text):
    clean=[]
    analysis_input = text
    print('Analysis result for input : ' + fix_decode(analysis_input))
    analysis_result = analyze(analysis_input)
    for a in analysis_result.results:
        best = a.best
        # for l in best.lemmas:
        lemmas = best.lemmas[0]
        if lemmas!="UNK":
#             print("Lemmas = " + lemmas)
            clean.append(lemmas) 
    clean = " ".join(clean)
    return clean

In [ ]:
def just_date(timex):
    timex = datetime.strptime(timex, '%Y-%m-%dT%H:%M:%S.%fZ')
    return timex.date()

In [ ]:
isimler=[] # turkish names
with open("isimler",encoding="utf-8") as f:
    for line in f:
        line=line.lower()
        isimler.append(line[:-1])
# print(isimler)

In [ ]:

def isimsil(text):
    temiz=[]
    text=text.split()
    for w in text:
        if w not in isimler:
            temiz.append(w)
    return " ".join(temiz)


In [ ]:

def replacing(text):

    if "blok" and "saat" not in text.split():
        return text
    else:
        return "nan"

    # if "sözkonusu" and "ulaşabilirsiniz" in text.split():
    #     text=text.replace("sözkonusu","[sözkonusu")
    #     text = text.replace("ulaşabilirsiniz", "ulaşabilirsiniz]")

    # return text

In [ ]:
def clean_text(text, remove_stopwords=True):
    # text=text.lower()
    text=re.sub(r'\[[^\)]*\)', ' ', text)
    text=re.sub(r'\[[^\]]*\]', ' ', text)
    text=re.sub(r'\|[^\|]*\|', ' ', text)
    text=re.sub(r'\#|[^\#]*\#', ' ', text)
    text = re.sub("[%s]" % re.escape(string.punctuation), "", text)  ## noktalama işaretlerini aldk
    text = re.sub("\w*\d\w*", "", text)

    if remove_stopwords:
        text=text.split()
        stops=set(stopwords.words("turkish"))
        text=[w for w in text if not w in stops and w!="" and w!=""and w!=""]
        text=" ".join(text)


    return text

In [ ]:
def delete_short(text):
    text=text.split()
    if len(text)>5:
        return " ".join(text)
    else:
        return ""

In [ ]:
df.content=df.content.astype("U")
df.date = df.date.apply(just_date)
df.content = df.content.apply(replacing)
df.content = df.content.apply(clean_text)
df.content = df.content.apply(delete_short)
df.content = df.content.apply(isimsil)

In [ ]:
df = df[["date", "content"]]
df.content.dropna(inplace=True,axis=0)


In [ ]:

df=df[df.content!=""]
df=df[df.content!="nan"]

print(df.head())

# df.to_csv("PEG_NEWS.CSV")
# df.to_csv("THY_NEWS.CSV")